<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W2/ungraded_labs/C1_W2_Lab_2_callbacks_traducida.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Semana 2: Implementación de Callbacks en TensorFlow utilizando el conjunto de datos MNIST

En el curso aprendiste a hacer la clasificación usando la moda MNIST, un conjunto de datos que contiene artículos de ropa. Hay otro conjunto de datos similar llamado MNIST que tiene elementos de escritura a mano - los dígitos del 0 al 9.

Escribe un clasificador MNIST que se entrene hasta el 99% de precisión y se detenga una vez alcanzado este umbral. En la clase has visto cómo se hace esto para la pérdida, pero aquí usarás la precisión en su lugar.

Algunas notas:
1. Tu red debe tener éxito en menos de 9 épocas.
2. Cuando alcance el 99% o más, debería imprimir la cadena "¡Ha alcanzado el 99% de precisión, por lo que se cancela el entrenamiento!" y detener el entrenamiento.
3. Si añades alguna variable adicional, asegúrate de utilizar los mismos nombres que los utilizados en la clase. Esto es importante para las firmas de las funciones (los parámetros y nombres) de las devoluciones de llamada.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

# Week 2: Implementing Callbacks in TensorFlow using the MNIST Dataset

In the course you learned how to do classification using Fashion MNIST, a data set containing items of clothing. There's another, similar dataset called MNIST which has items of handwriting -- the digits 0 through 9.

Write an MNIST classifier that trains to 99% accuracy and stops once this threshold is achieved. In the lecture you saw how this was done for the loss but here you will be using accuracy instead.

Some notes:
1. Your network should succeed in less than 9 epochs.
2. When it reaches 99% or greater it should print out the string "Reached 99% accuracy so cancelling training!" and stop training.
3. If you add any additional variables, make sure you use the same names as the ones used in the class. This is important for the function signatures (the parameters and names) of the callbacks.

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

## Cargar e inspeccionar los datos

Comience por cargar los datos. Hay que tener en cuenta un par de cosas:

- El archivo `mnist.npz` ya está incluido en el espacio de trabajo actual bajo el directorio `data`. Por defecto el `load_data` de Keras acepta una ruta relativa a `~/.keras/datasets` pero en este caso se almacena en otro lugar, por lo que es necesario especificar la ruta completa.

- El programa `load_data` devuelve los conjuntos de entrenamiento y de prueba en forma de tuplas `(x_train, y_train), (x_test, y_test)` pero en este ejercicio sólo necesitarás el conjunto de entrenamiento por lo que puedes ignorar la segunda tupla.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Load and inspect the data

Begin by loading the data. A couple of things to notice:

- The file `mnist.npz` is already included in the current workspace under the `data` directory. By default the `load_data` from Keras accepts a path relative to `~/.keras/datasets` but in this case it is stored somewhere else, as a result of this, you need to specify the full path.

- `load_data` returns the train and test sets in the form of the tuples `(x_train, y_train), (x_test, y_test)` but in this exercise you will be needing only the train set so you can ignore the second tuple.

In [3]:
# Cargar los datos

# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Añadir data/mnist.npz a la ruta anterior para obtener la ruta completa
data_path = os.path.join(current_dir, "data/mnist.npz")

# Descartar el conjunto de pruebas
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path=data_path)
        
# Normalizar los valores de los píxeles
x_train = x_train / 255.0

Ahora observa la forma de los datos de entrenamiento:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Now take a look at the shape of the training data:

In [4]:
data_shape = x_train.shape

print(f"There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})")

There are 60000 examples with shape (28, 28)


## Definiendo su callback

Ahora es el momento de crear tu propio callback personalizado. Para ello completa la clase `myCallback` y el método `on_epoch_end` en la celda de abajo. Si necesitas orientación sobre cómo proceder, consulta este [enlace](https://www.tensorflow.org/guide/keras/custom_callback).

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Defining your callback

Now it is time to create your own custom callback. For this complete the `myCallback` class and the `on_epoch_end` method in the cell below. If you need some guidance on how to proceed, check out this [link](https://www.tensorflow.org/guide/keras/custom_callback).

In [14]:
# GRADED CLASS: myCallback
### START CODE HERE

# Recuerda heredar de la clase correcta
class myCallback(tf.keras.callbacks.Callback):
        # Define la firma de la función correcta para on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Detener el entrenamiento una vez que se cumpla la condición anterior
                self.model.stop_training = True

### END CODE HERE


## Crea y entrena tu modelo

Ahora que has definido tu callback es el momento de completar la función `train_mnist` de abajo. 

**Debes configurar tu modelo para que se entrene durante 10 épocas y la llamada de retorno debe dispararse antes de la novena época para que apruebes esta tarea.**

**Sugerencia:**
- Siéntete libre de probar la arquitectura para la red neuronal que consideres oportuna pero en caso de que necesites ayuda extra puedes consultar una arquitectura que funciona bastante bien al final de este cuaderno.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Create and train your model

Now that you have defined your callback it is time to complete the `train_mnist` function below. 

**You must set your model to train for 10 epochs and the callback should fire before the 9th epoch for you to pass this assignment.**

**Hint:**
- Feel free to try the architecture for the neural network that you see fit but in case you need extra help you can check out an architecture that works pretty well at the end of this notebook.

In [15]:
# GRADED FUNCTION: train_mnist
def train_mnist(x_train, y_train):

    ### START CODE HERE
    
    #  Instanciar la clase callback
    callbacks = myCallback()
    
    # Definir el modelo
    model = tf.keras.models.Sequential([ 
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation=tf.nn.relu),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ]) 

    # Compilar el modelo
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
    
    # Ajustar el modelo durante 10 épocas añadiendo los callbacks
    # y guardar el historial de entrenamiento
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    ### END CODE HERE

    return history

Llama al `train_mnist` pasando los parámetros adecuados para obtener el historial de entrenamiento:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Call the `train_mnist` passing in the appropiate parameters to get the training history:

In [16]:
hist = train_mnist(x_train, y_train)

Epoch 1/10
1875/1875 [==============================] - 9s 4ms/step - loss: 0.1992 - accuracy: 0.9403
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0795 - accuracy: 0.9762
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0528 - accuracy: 0.9833
Epoch 4/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0366 - accuracy: 0.9880
Epoch 5/10
1870/1875 [============================>.] - ETA: 0s - loss: 0.0274 - accuracy: 0.9912
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0274 - accuracy: 0.9912


Si ves el mensaje "Alcanzado el 99% de precisión, por lo que se cancela el entrenamiento" impreso después de menos de 9 épocas, significa que tu devolución de llamada ha funcionado como se esperaba. 

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

If you see the message `Reached 99% accuracy so cancelling training!` printed out after less than 9 epochs it means your callback worked as expected. 

## ¿Necesitas más ayuda?

Ejecute la siguiente celda para ver una arquitectura que funciona bien para el problema en cuestión:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Need more help?

Run the following cell to see an architecture that works well for the problem at hand:

In [ ]:
# LE RECOMENDAMOS ENCARECIDAMENTE QUE PRUEBE PRIMERO SUS PROPIAS ARQUITECTURAS
# Y SÓLO EJECUTE ESTA CELDA SI DESEA VER UNA RESPUESTA

import base64

encoded_answer = "CiAgIC0gQSBGbGF0dGVuIGxheWVyIHRoYXQgcmVjZWl2ZXMgaW5wdXRzIHdpdGggdGhlIHNhbWUgc2hhcGUgYXMgdGhlIGltYWdlcwogICAtIEEgRGVuc2UgbGF5ZXIgd2l0aCA1MTIgdW5pdHMgYW5kIFJlTFUgYWN0aXZhdGlvbiBmdW5jdGlvbgogICAtIEEgRGVuc2UgbGF5ZXIgd2l0aCAxMCB1bml0cyBhbmQgc29mdG1heCBhY3RpdmF0aW9uIGZ1bmN0aW9uCg=="
encoded_answer = encoded_answer.encode('ascii')
answer = base64.b64decode(encoded_answer)
answer = answer.decode('ascii')

print(answer)

**Felicitaciones por terminar la tarea de esta semana.**

Has implementado con éxito una devolución de llamada que te da más control sobre el bucle de entrenamiento de tu modelo. Buen trabajo.

**Sigue así.**

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

**Congratulations on finishing this week's assignment!**

You have successfully implemented a callback that gives you more control over the training loop for your model. Nice job!

**Keep it up!**